In [3]:
# multivariate lstm example
from numpy import array
from numpy import hstack
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.metrics import r2_score # R square

In [5]:
excelFile = 'Preprocessed Data.xlsx'
#读取数据，指定日期列为指标，Pandas自动将“日期”列识别为Datetime格式
data = pd.read_excel(excelFile,sheet_name=0)
data['Date '] = data['Date '].apply(lambda x:x.strftime('%Y-%m-%d'))
data['Time (hh:mm)'] = data['Time (hh:mm)'].apply(lambda x:x.strftime('%H:%M:%S'))
data['datetime'] =pd.to_datetime(data['Date '] + ' ' + data['Time (hh:mm)'])
data.drop(list(data)[0:2],axis=1,inplace=True)
# data=data.set_index('datetime')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Preprocessed Data.xlsx'

In [3]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [44]:
# define input sequence
in_seq1 = data['Ambient temperature (°C)'].values
in_seq2 = data['Global plane of array irradiance (W/m²)'].values
in_seq3 = data['m-Si module temperature (°C)'].values
in_seq4 = data['DC power of m-Si (W)'].values
out_seq = data['DC power of m-Si (W)'].values
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2,in_seq3,in_seq4, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out =117 , 3
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# flatten input
n_input = X.shape[1] * X.shape[2]
n_features = X.shape[2]
# X = X.reshape((X.shape[0], n_input))
X_train = X[:int(len(X)*0.7)]
y_train = y[:int(len(X)*0.7)]
X_test = X[int(len(X)*0.7):]
y_test = y[int(len(X)*0.7):]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu',  input_shape=(n_steps_in, n_features)))
# model.add(LSTM(4, activation='relu'))
# model.add(Dense(100))
# model.add(Dense(50))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, y_train, epochs=300,verbose=1)
# demonstrate prediction
# x_input = array([[70, 75], [80, 85], [90, 95]])
# x_input = x_input.reshape((1, n_input))
# yhat = model.predict(x_input, verbose=0)
# print(yhat)

Epoch 1/300
112/112 [==============================] - 4s 11ms/step - loss: 1303667.4320
Epoch 2/300
112/112 [==============================] - 1s 10ms/step - loss: 289405.7027
Epoch 3/300
112/112 [==============================] - 1s 9ms/step - loss: 188314.5961
Epoch 4/300
112/112 [==============================] - 1s 9ms/step - loss: 161499.4405
Epoch 5/300
112/112 [==============================] - 1s 9ms/step - loss: 122669.8055
Epoch 6/300
112/112 [==============================] - 1s 9ms/step - loss: 106530.7796
Epoch 7/300
112/112 [==============================] - 1s 9ms/step - loss: 64113.4514
Epoch 8/300
112/112 [==============================] - 1s 9ms/step - loss: 119705.1148
Epoch 9/300
112/112 [==============================] - 1s 10ms/step - loss: 58168.4621
Epoch 10/300
112/112 [==============================] - 1s 9ms/step - loss: 49298.0272
Epoch 11/300
112/112 [==============================] - 1s 9ms/step - loss: 47652.4073
Epoch 12/300
112/112 [===================

112/112 [==============================] - 1s 9ms/step - loss: 10355.3052
Epoch 187/300
112/112 [==============================] - 1s 9ms/step - loss: 9205.3087
Epoch 188/300
112/112 [==============================] - 1s 9ms/step - loss: 10205.6453
Epoch 189/300
112/112 [==============================] - 1s 9ms/step - loss: 11112.2601
Epoch 190/300
112/112 [==============================] - 1s 9ms/step - loss: 11030.0529
Epoch 191/300
112/112 [==============================] - 1s 9ms/step - loss: 10766.6608
Epoch 192/300
112/112 [==============================] - 1s 10ms/step - loss: 11249.2457
Epoch 193/300
112/112 [==============================] - 1s 9ms/step - loss: 10961.7038
Epoch 194/300
112/112 [==============================] - 1s 9ms/step - loss: 11508.5142
Epoch 195/300
112/112 [==============================] - 1s 9ms/step - loss: 11245.3312
Epoch 196/300
112/112 [==============================] - 1s 9ms/step - loss: 12299.0239
Epoch 197/300
112/112 [=======================

112/112 [==============================] - 1s 9ms/step - loss: 9448.7632
Epoch 280/300
112/112 [==============================] - 1s 9ms/step - loss: 11579.2976
Epoch 281/300
112/112 [==============================] - 1s 9ms/step - loss: 11386.5524
Epoch 282/300
112/112 [==============================] - 1s 10ms/step - loss: 9674.8923
Epoch 283/300
112/112 [==============================] - 1s 9ms/step - loss: 13118.6651
Epoch 284/300
112/112 [==============================] - 1s 9ms/step - loss: 10952.7845
Epoch 285/300
112/112 [==============================] - 1s 11ms/step - loss: 10841.4298
Epoch 286/300
112/112 [==============================] - 1s 11ms/step - loss: 12691.1222
Epoch 287/300
112/112 [==============================] - 1s 10ms/step - loss: 11735.4536
Epoch 288/300
112/112 [==============================] - 1s 9ms/step - loss: 11431.9848
Epoch 289/300
112/112 [==============================] - 1s 10ms/step - loss: 9834.4338
Epoch 290/300
112/112 [=====================

In [50]:
yhat = model.predict(X_test, verbose=0)
yhat

array([[1711.7104 , 1696.6497 , 1699.6993 ],
       [1712.1466 , 1698.3279 , 1702.0631 ],
       [1723.6337 , 1707.6174 , 1709.5948 ],
       ...,
       [ 383.98666,  368.53622,  380.3894 ],
       [ 344.2996 ,  352.89944,  391.5462 ],
       [ 316.23843,  347.56784,  408.97418]], dtype=float32)

In [51]:
y_test

array([[1697., 1698., 1706.],
       [1698., 1706., 1719.],
       [1706., 1719., 1728.],
       ...,
       [ 384.,  341.,  310.],
       [ 341.,  310.,  274.],
       [ 310.,  274.,  241.]])

In [52]:
y_t = pd.DataFrame(y_test)
y_p = pd.DataFrame(yhat)

In [53]:
def evaluate(y_true,y_pred):
    SSE = sum((y_true-y_pred)**2)
    MSE = SSE/len(y_true)
    RMSE = MSE**(1/2)
    MAE = sum(abs(y_true-y_pred))/len(y_true)
    MAPE = sum(abs(y_true-y_pred)/y_true)/len(y_true)*100
    SD = (sum((abs(y_true-y_pred)-sum(abs(y_true-y_pred))/len(y_true))**2)/len(y_true))**(1/2)
    r2s = r2_score(y_true, y_pred)
    print('MSE: ', MSE, '\n', 'RMSE: ', RMSE, '\n','MAE: ', MAE, '\n','MAPE: ', MAPE, '\n','SD: ', SD, '\n', 'R square: ', r2s)

In [56]:
evaluate(y_t[2],y_p[2])

MSE:  14713.700159245122 
 RMSE:  121.30004187651842 
 MAE:  57.01136620693519 
 MAPE:  7.077160998262558 
 SD:  107.06728857341949 
 R square:  0.9316551356474456
